## Аналіз A/B тесту

Маємо проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це класична гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку. У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми проаналізуємо вплив на утримання (retention) гравців. Тобто хочемо зрозуміти чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Змінні в даних наступні:

- userid - унікальний номер, який ідентифікує кожного гравця.
- version - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- sum_gamerounds - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- retention_1 - чи через 1 день після встановлення гравець повернувся і почав грати?
- retention_7 - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/drive/MyDrive/data/cookie_cats.csv')

In [4]:
df["retention_7"] = df["retention_7"].astype(int)

In [5]:
clean_data = df[['version', 'retention_7']]

In [6]:
control_results = clean_data[clean_data['version'] == 'gate_30']
treatment_results = clean_data[clean_data['version'] == 'gate_40']
print ('середнє значення конверсії контрольної групи:', control_results['retention_7'].mean(),
       'середнє значення конверсії експериментальної групи:', treatment_results['retention_7'].mean())

середнє значення конверсії контрольної групи: 0.19020134228187918 середнє значення конверсії експериментальної групи: 0.18200004396667327


**H0:** значення retention_7 не залежить від версії гри

**H1:** версія гри впливає на значення retention_7

2. Перевірте з допомогою z-тесту аналогічно до прикладу в лекції, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для двох вибірок. Виведіть результат у форматі:
```
z statistic: ...
p-value: ...
Довірчий інтервал 95% для групи control: [..., ...]
Довірчий інтервал 95% для групи treatment: [..., ...]
```
де замість `...` - обчислені значення. В якості висновка дайте відповідь на два питання:  
    1. чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
    2. чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  
    
Зверніть увагу, в такому і схожому завданнях ми використовуєм `proportion` Z-тест. Це тому що в нас залежна змінна має бінарне значення (повернеться аби ні користувач, чи клікне або ні користувач в інших ситуаціях - всього два можливих значення в змінної: 0/1, True/False ). Якщо б ми вимірювали скажімо чи є стат. значущою різниця між вагою чоловіків і жінок в певній вибірці, ми б використовувавли функцію `statsmodels.stats.ztest`, бо залежна змінна `вага` є неперервною (тип float, замість типу int чи bool і тільки двох можливих значень).

In [7]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [8]:
n_con = control_results.count()
n_treat = treatment_results.count()

In [17]:
successes = [control_results['retention_7'].sum(), treatment_results['retention_7'].sum()]
nobs = [control_results['retention_7'].count(), treatment_results['retention_7'].count()]

In [27]:
z_stat, pval = proportions_ztest(successes, nobs=nobs)
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes, nobs=nobs, alpha=0.05)

print(f'z statistic: {z_stat:.2f}')
print(f'p-value: {pval:.3f}')
print(f'Довірчий інтервал 95% для групи control: [{lower_con:.3f}, {upper_con:.3f}]')
print(f'Довірчий інтервал 95% для групи treatment: [{lower_treat:.3f}, {upper_treat:.3f}]')

alpha = 0.05
if p < alpha:
    print("Відхиляємо нульову гіпотезу: є статистично значуща залежність між версією гри і утриманням на 7-й день.")
else:
    print("Не відхиляємо нульову гіпотезу: статистично значущої залежності немає.")

z statistic: 3.16
p-value: 0.002
Довірчий інтервал 95% для групи control: [0.187, 0.194]
Довірчий інтервал 95% для групи treatment: [0.178, 0.186]
Відхиляємо нульову гіпотезу: є статистично значуща залежність між версією гри і утриманням на 7-й день.


*p-value = 0.002 < 0.05, тому відхиляємо нульову гіпотезу. Версія гри дійсно впливає на утримання користувачів на 7-й день, але не так, як очікувалось початково: перенесення воріт на 40-й рівень зменшило рівень утримання гравців.*

*Довірчі інтервали для групи control [0.187, 0.194] та treatment [0.178, 0.186] не перетинаються.*
*Це свідчить про те, що різниця між утриманням у цих групах не є випадковою і підтверджує наявність значущого ефекту.*

3. Є ще один тип тестів, який використовується для бінарної метрики як от "зробить юзер дію, чи ні" - тест [**Хі-квадрат**](https://www.bmj.com/about-bmj/resources-readers/publications/statistics-square-one/8-chi-squared-tests) (ще одне [пояснення](https://www.scribbr.com/statistics/chi-square-tests/) тесту з прикладами). В нього інші гіпотези Н0 і Н1 на відміну від z- та t-тестів. А також цей тест можна використовувати, якщо в нас більше за 2 досліджувані групи, тобто в нас не А/В тест, а А/B/C/D, наприклад.  

В **z- та t-тестах** (які відрізняються тим, що ми в першому не знаємо дисперсію генеральної сукупності, але якщо в нас великий набір даних, то ці два тести дають дуже схожі результати) **ми перевіряємо, чи є різниця у середніх показниках по групам користувачів**.  

А в **тесті Хі-квадрат ми перевіряємо чи є звʼязок між групою користувача і тим, чи він зробить цікаву нам дію**. Це ніби дослідження одного і того самого, але дещо різними способами. Для перевірки, можна виконувати кілька тестів (особливо, якщо один дає якийсь непереконливий результат типу р-значення 0.07 - наче і fail to regect H0 на рівні стат значущості 5%, але цікаво, що скажуть інші тести), тож, зробимо і ми тест хі-квадрат та порівняємо його результат з z-тестом.

Про різницю між тестами можна почитати ще [тут](https://stats.stackexchange.com/a/178860) - це просто пояснення користувача стековерфлоу, але там розумні люди сидять.

Для проведення хі-квадрат тесту скористаємось функцією з `scipy.stats` `chi2_contingency` для обчислення статистики хі-квадрат і р-значення для перевірки конкретної гіпотези. У цю функцію вам треба передати таблицю 2х2: кількість випадків для кожної версії гри і значення `retention_7`.

**Задача**: виконайте тест хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та тим, чи зайде гравець на 7ий день після встановлення гри.
Тут гіпотези наступні
- Н0: значення retention_7 не залежить від версії гри
- Н1: є залежність між версією гри і значенням retention_7

Виведіть p-значення та зробіть висновок.


In [14]:
from scipy.stats import chi2_contingency

In [28]:
contingency_table = pd.crosstab(df['version'], df['retention_7'])
chi2, p, dof, expected = chi2_contingency(contingency_table)
print(f"Chi2-статистика: {chi2}")
print(f"p-значення: {p}")
print(f"Ступені свободи: {dof}")
print("Очікувані частоти:")
print(expected)

alpha = 0.05
if p < alpha:
    print("Відхиляємо нульову гіпотезу: є статистично значуща залежність між версією гри і утриманням на 7-й день.")
else:
    print("Не відхиляємо нульову гіпотезу: статистично значущої залежності немає.")

Chi2-статистика: 9.959086799559167
p-значення: 0.0016005742679058301
Ступені свободи: 1
Очікувані частоти:
[[36382.90257127  8317.09742873]
 [37025.09742873  8463.90257127]]
Відхиляємо нульову гіпотезу: є статистично значуща залежність між версією гри і утриманням на 7-й день.
